# Homework 07:  Topic Modeling with Latent Semantic Analysis

In [1]:
import numpy as np
import string
import math
from collections import Counter

Latent Semantic Analysis (LSA) is a method for finding latent similarities between documents treated as a bag of words by using a low rank approximation. It is used for document classification, clustering and retrieval. For example, LSA can be used to search for prior art given a new patent application. In this homework, we will implement a small library for simple latent semantic analysis as a practical example of the application of SVD. The ideas are very similar to PCA. SVD is also used in recommender systems in an similar fashion (for an SVD-based recommender system library, see [Surpise](http://surpriselib.com)). 

We will implement a toy example of LSA to get familiar with the ideas. If you want to use LSA or similar methods for statistical language analysis, the most efficient Python libraries are probably [gensim](https://radimrehurek.com/gensim/) and [spaCy](https://spacy.io) - these also provide an online algorithm - i.e. the training information can be continuously updated. Other useful functions for processing natural language can be found in the [Natural Language Toolkit](http://www.nltk.org/).

**Note**: The SVD from scipy.linalg performs a full decomposition, which is inefficient since we only need to decompose until we get the first k singluar values. If the SVD from `scipy.linalg` is too slow, please use the `sparsesvd` function from the [sparsesvd](https://pypi.python.org/pypi/sparsesvd/) package to perform SVD instead.  You can install in the usual way with 
```
!pip install sparsesvd
```

Then import the following
```python
from sparsesvd import sparsesvd 
from scipy.sparse import csc_matrix 
```

and use as follows
```python
sparsesvd(csc_matrix(M), k=10)
```

**Exercise 1 (20 points)**.  Calculating pairwise distance matrices.

Suppose we want to construct a distance matrix between the rows of a matrix. For example, given the matrix 

```python
M = np.array([[1,2,3],[4,5,6]])
```

the distance matrix using Euclidean distance as the measure would be
```python
[[ 0.000  1.414  2.828]
 [ 1.414  0.000  1.414]
 [ 2.828  1.414  0.000]] 
```
if $M$ was a collection of column vectors.

Write a function to calculate the pairwise-distance matrix given the matrix $M$ and some arbitrary distance function. Your functions should have the following signature:
```
def func_name(M, distance_func):
    pass
```

0. Write a distance function for the Euclidean, squared Euclidean and cosine measures.
1. Write the function using looping for M as a collection of row vectors.
2. Write the function using looping for M as a collection of column vectors.
3. Write the function using broadcasting for M as a collection of row vectors.
4. Write the function using broadcasting for M as a collection of column vectors. 

For 3 and 4, try to avoid using transposition (but if you get stuck, there will be no penalty for using transposition).

We start by defining M exactly as it is in the instructions above (which makes it a collection of row vectors)

In [2]:
M = np.array([[1,2,3],[4,5,6]])
M

array([[1, 2, 3],
       [4, 5, 6]])

Write a distance function for the Euclidean, squared Euclidean, and cosine measures

In [3]:
def sq_euclid(v1, v2):
    """Calculate pairwise squared Euclidean distances between two input vectors"""
    return np.square(v1 - v2).sum()

In [4]:
def euclid(v1, v2):
    """Calculate pairwise Euclidean distances between two input vectors"""
    return np.sqrt(sq_euclid(v1, v2))

In [9]:
def cos_dist(v1, v2):
    return 1 - (v1 @ v2.T) / (np.linalg.norm(v1) * np.linalg.norm(v2))

Write the function using looping for M as a collection of row vectors

In [10]:
def part2(M, distance_func):
    """Compute specified distance metric between rows of M, where M is specified as a collection of row vectors"""
    result = np.zeros(shape = (M.shape[0], M.shape[0]))
    for i in range(M.shape[0]):
        for j in range(i + 1, M.shape[0]):
            result[i, j] = distance_func(M[i, :], M[j, :])
    return result + result.T

In [11]:
# Euclidean distance
print("Euclidean distance")
print(np.round(part2(M, euclid), decimals = 3))

# Squared euclidean distance
print("Squared euclidean distance")
print(np.round(part2(M, sq_euclid), decimals = 3))

# Cosine distance
print("Cosine distance")
print(np.round(part2(M, cos_dist), decimals = 3))

Euclidean distance
[[0.    5.196]
 [5.196 0.   ]]
Squared euclidean distance
[[ 0. 27.]
 [27.  0.]]
Cosine distance
[[0.    0.025]
 [0.025 0.   ]]


Write the function using looping for M as a collection of column vectors

*A solution using transposition* 

In [12]:
def part3(M, distance_func):
    """Compute specified distance metric between rows of M, where M is specified as a collection of column vectors"""
    return part2(M.T, distance_func)

In [13]:
# Euclidean distance
print("Euclidean distance")
print(np.round(part3(M, euclid), decimals = 3))

# Squared euclidean distance
print("Squared euclidean distance")
print(np.round(part3(M, sq_euclid), decimals = 3))

# Cosine distance
print("Cosine distance")
print(np.round(part3(M, cos_dist), decimals = 3))

Euclidean distance
[[0.    1.414 2.828]
 [1.414 0.    1.414]
 [2.828 1.414 0.   ]]
Squared euclidean distance
[[0. 2. 8.]
 [2. 0. 2.]
 [8. 2. 0.]]
Cosine distance
[[0.    0.009 0.024]
 [0.009 0.    0.003]
 [0.024 0.003 0.   ]]


*A solution without transposition*

In [14]:
def part3_2(M, distance_func):
    """Compute specified distance metric between rows of M, where M is specified as a collection of column vectors"""
    result = np.zeros(shape = (M.shape[1], M.shape[1]))
    for i in range(M.shape[1]):
        for j in range(i + 1, M.shape[1]):
            result[i, j] = distance_func(M[:, i], M[:, j])
    return result + result.T

In [15]:
# Euclidean distance
print("Euclidean distance")
print(np.round(part3_2(M, euclid), decimals = 3))

# Squared euclidean distance
print("Squared euclidean distance")
print(np.round(part3_2(M, sq_euclid), decimals = 3))

# Cosine distance
print("Cosine distance")
print(np.round(part3_2(M, cos_dist), decimals = 3))

Euclidean distance
[[0.    1.414 2.828]
 [1.414 0.    1.414]
 [2.828 1.414 0.   ]]
Squared euclidean distance
[[0. 2. 8.]
 [2. 0. 2.]
 [8. 2. 0.]]
Cosine distance
[[0.    0.009 0.024]
 [0.009 0.    0.003]
 [0.024 0.003 0.   ]]


Write the function using broadcasting for M as a collection of row vectors

In [82]:
test = (M[None, :] - M[:, None])
test

array([[[ 0,  0,  0],
        [ 3,  3,  3]],

       [[-3, -3, -3],
        [ 0,  0,  0]]])

In [18]:
test2 = (M.T[None, :] - M.T[:, None])
test2

array([[[ 0,  0],
        [ 1,  1],
        [ 2,  2]],

       [[-1, -1],
        [ 0,  0],
        [ 1,  1]],

       [[-2, -2],
        [-1, -1],
        [ 0,  0]]])

In [109]:
def part4(M, distance_func):
    """Compute specified distance metric between rows of M (using broadcasting), 
    
    where M is specified as a collection of column vectors"""
    
    # Calculate rows differences using broadcasting
    dists = M[None, :] - M[:, None]
    
    # Calculate specified distance matric
    result = np.zeros(shape = (M.shape[0], M.shape[0]))
    for i in range(dists.shape[0]):
        for j in range(i + 1, dists.shape[0]):
            result[i, j] = distance_func(dists[i, 0, :], dists[i, j, :])
    return result + result.T

In [112]:
# There's a bug for cos_dist
part4(M, cos_dist)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([[ 0., nan],
       [nan,  0.]])

In [12]:
np.round(np.sqrt(np.square(M.T[None, :] - M.T[:, None]).sum(axis = -1)), decimals = 3)

array([[0.   , 1.414, 2.828],
       [1.414, 0.   , 1.414],
       [2.828, 1.414, 0.   ]])

In [13]:
np.round(np.sqrt(np.square(M[None, :] - M[:, None]).sum(axis = -1)), decimals = 3)

array([[0.   , 5.196],
       [5.196, 0.   ]])

**Exercise 2 (20 points)**. Write 3 functions to calculate the term frequency (tf), the inverse document frequency (idf) and the product (tf-idf). Each function should take a single argument `docs`, which is a dictionary of (key=identifier, value=document text) pairs, and return an appropriately sized array. Convert '-' to ' ' (space), remove punctuation, convert text to lowercase and split on whitespace to generate a collection of terms from the document text.

- tf = the number of occurrences of term $i$ in document $j$
- idf = $\log \frac{n}{1 + \text{df}_i}$ where $n$ is the total number of documents and $\text{df}_i$ is the number of documents in which term $i$ occurs.

Print the table of tf-idf values for the following document collection

```
s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = {'s1': s1, 's2': s2, 's3': s3, 's4': s4}
```

In [6]:
def format_text(doc):
    """Formats text from a single document"""
    doc = doc.translate(str.maketrans('-', ' ')).lower()
    return doc.translate(str.maketrans('', '', string.punctuation)).split(' ')

In [7]:
def calc_tf(docs):
    """Calculate term frequency for all documents in the input"""
    
    # Find all unique words in docs
    words = np.unique(format_text(str(list(docs.values()))))
    
    # Initialize output array and column counter
    result = np.zeros(shape = (len(words), len(list(docs.keys()))))
    col = 0
    
    # For each document, count words, and assign to tf-matrix
    for doc in docs:
        counted_words = Counter(format_text(docs[doc]))
        for word in counted_words.elements():
            row = np.where(words == word)
            result[row, col] = counted_words[word]
        col += 1
    
    # Return result
    return result

In [21]:
def calc_idf(docs):
    """Calculate inverse document frequency"""
    
    # Find all unique words in docs
    words = np.unique(format_text(str(list(docs.values()))))
    
    # Initialize output array
    df_i = np.zeros(len(words))
    
    # Determine number of documents in input
    n = len(list(docs.keys()))
    
    # Calculate df_i
    for doc in docs:
        words_unique = np.unique(format_text(docs[doc]))
        for word in words_unique:
            index = np.where(words == word)
            df_i[index] += 1
    
    # Return idf
    return (np.log(n / (1 + df_i))).reshape(-1, 1)

In [9]:
calc_tf(docs)

array([[1., 1., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 1.],
       [1., 1., 0., 0.],
       [0., 4., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 1., 3., 5.],
       [0., 0., 0., 1.]])

In [22]:
calc_idf(docs)

array([[ 0.28768207],
       [ 0.28768207],
       [ 0.28768207],
       [ 0.28768207],
       [ 0.69314718],
       [ 0.69314718],
       [ 0.69314718],
       [ 0.69314718],
       [ 0.69314718],
       [ 0.69314718],
       [-0.22314355],
       [ 0.69314718]])

In [4]:
# Create dictionary for testing
s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = {'s1': s1, 's2': s2, 's3': s3, 's4': s4}

**Exercise 3 (20 points)**. 

1. Write a function that takes a matrix $M$ and an integer $k$ as arguments, and reconstructs a reduced matrix using only the $k$ largest singular values. Use the `scipy.linagl.svd` function to perform the decomposition. This is the least squares approximation to the matrix $M$ in $k$ dimensions.

2. Apply the function you just wrote to the following term-frequency matrix for a set of $9$ documents using $k=2$ and print the reconstructed matrix $M'$.
```
M = np.array([[1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 2, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1]])
```

3. Calculate the pairwise correlation matrix for the original matrix M and the reconstructed matrix using $k=2$ singular values (you may use [scipy.stats.spearmanr](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html) to do the calculations). Consider the first 5 sets of documents as one group $G1$ and the last 4 as another group $G2$ (i.e. first 5 and last 4 columns). What is the average within group correlation for $G1$, $G2$ and the average cross-group correlation for G1-G2 using either $M$ or $M'$. (Do not include self-correlation in the within-group calculations.).

**Exercise 4 (40 points)**. Clustering with LSA

1. Begin by loading a PubMed database of selected article titles using 'pickle'. With the following:
```import pickle
docs = pickle.load(open('pubmed.pic', 'rb'))```

    Create a tf-idf matrix for every term that appears at least once in any of the documents. What is the shape of the tf-idf matrix? 

2. Perform SVD on the tf-idf matrix to obtain $U \Sigma V^T$ (often written as $T \Sigma D^T$ in this context with $T$ representing the terms and $D$ representing the documents). If we set all but the top $k$ singular values to 0, the reconstructed matrix is essentially $U_k \Sigma_k V_k^T$, where $U_k$ is $m \times k$, $\Sigma_k$ is $k \times k$ and $V_k^T$ is $k \times n$. Terms in this reduced space are represented by $U_k \Sigma_k$ and documents by $\Sigma_k V^T_k$. Reconstruct the matrix using the first $k=10$ singular values.

3. Use agglomerative hierarchical clustering with complete linkage to plot a dendrogram and comment on the likely number of  document clusters with $k = 100$. Use the dendrogram function from [SciPy ](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.cluster.hierarchy.dendrogram.html).

4. Determine how similar each of the original documents is to the new document `data/mystery.txt`. Since $A = U \Sigma V^T$, we also have $V = A^T U S^{-1}$ using orthogonality and the rule for transposing matrix products. This suggests that in order to map the new document to the same concept space, first find the tf-idf vector $v$ for the new document - this must contain all (and only) the terms present in the existing tf-idx matrix. Then the query vector $q$ is given by $v^T U_k \Sigma_k^{-1}$. Find the 10 documents most similar to the new document and the 10 most dissimilar. 

**Notes on the Pubmed articles**

These were downloaded with the following script.

```python
from Bio import Entrez, Medline
Entrez.email = "YOUR EMAIL HERE"
import cPickle

try:
    docs = cPickle.load(open('pubmed.pic'))
except Exception, e:
    print e

    docs = {}
    for term in ['plasmodium', 'diabetes', 'asthma', 'cytometry']:
        handle = Entrez.esearch(db="pubmed", term=term, retmax=50)
        result = Entrez.read(handle)
        handle.close()
        idlist = result["IdList"]
        handle2 = Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text")
        result2 = Medline.parse(handle2)
        for record in result2:
            title = record.get("TI", None)
            abstract = record.get("AB", None)
            if title is None or abstract is None:
                continue
            docs[title] = '\n'.join([title, abstract])
            print title
        handle2.close()
    cPickle.dump(docs, open('pubmed.pic', 'w'))
docs.values()
```